In [3]:
import xlrd
from pathlib import Path
import pandas as pd
import numpy as np
from numba import decorators
import librosa

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
# read in the data file
# Give the location of the file 

df = pd.read_excel(r'data/data.xlsx', sheet_name='normalized')
# print(df)

In [6]:
## LOADING IN DATASETS

dataset = Path.cwd().joinpath("SongEmotionDataset")
datasheet = Path.cwd().joinpath("data") # for csua

#emotion labels
label_loc = datasheet.joinpath("data.xlsx")
wb = xlrd.open_workbook(label_loc) 
sheet = wb.sheet_by_index(3) #1 for 9 columns, 2 for 5 columns, 3 for normalized 8 columns

#emotion arr
emotions = ["solemnity", "tenderness", "nostalgia", "calmness", "power", "joyful activation", "tension", "sadness"]
# emotions = ["amazement", "calmness", "power", "joyful activation", "sadness"]
# emotions = ["amazement", "solemnity", "tenderness", "nostalgia", "calmness", "power", "joyful activation", "tension", "sadness"]

In [7]:
train_percentage = 0.8

train_song = []
test_song = []
train_emotion = []
test_emotion = []

row_indexes = np.arange(1,226)
np.random.shuffle(row_indexes)

def get_data(indexes):
    print(indexes)
    song = []
    emotion = []
    
    totals = torch.zeros(len(emotions), device=device).float()
    for x in indexes:
        row = torch.tensor([sheet.cell_value(x, j) for j in range(3, 11)], device=device).float()
        trackid = int(sheet.cell_value(x, 0))
        song.append(dataset.joinpath("{}.mp3".format(trackid)))
        emotion.append(row)
            
    return song, emotion
    
song, emotion = get_data(row_indexes)
print(song)
print(emotion)
# test_song, test_emotion = get_data(test_indexes)

train_song = [song[i] for i in range(len(song)) if i < len(song)*train_percentage]
test_song = [song[i] for i in range(len(song)) if i >= len(song)*train_percentage]

train_emotion = [emotion[i] for i in range(len(emotion)) if i < len(emotion)*train_percentage]
test_emotion = [emotion[i] for i in range(len(emotion)) if i >= len(emotion)*train_percentage]

# num_maxes = [0 for _ in emotions]

# for row in emotion:
#     i = torch.argmax(row)
#     print(i)
#     num_maxes[i] += 1
    
# print(num_maxes)

[120 132  39   3  48 134 191 101  33 148  90 160 208  79 121  69  10 189
  72 109 220 185 151  47  40 164 144 147 142 178 217 140 150   9  96 194
 223 165 143  61 149  53 158 193  62 115  64 137 152   6 153  74  54 145
  35  77  93 116 192 212  59  97 141  65  84 123  20 105  11 175 216 222
  98 172 119  87  41 225 201 135 179  71  67 133 218 176  25 168 177 202
  78  26  80 173 190  73  23  88  60 118 199   5  56  99 215   8 113 129
 138  81 188  49  29  36 206  31  16 155   2 111 112 205 181 162 209 102
  76  46  86 117 128  44 174  82  50  12 103  13   4 169  37 124 157 154
  85 211 127  18 180 159  95 213 224   1  52 210  21  38  17  55  70 187
  14 126 136  27 100 163  32  91  68 130  66 108 219  30  89 186 114 122
 204 200  94 197 184 167 183  19 182   7 170  51 171 196 106 221 110  92
 195  75 161  15  43  22  24  63  83  28  42  34 166 146 125 131  57 107
 214 198 156  58  45 203 207 104 139]
[PosixPath('/home/czf/memories/SongEmotionModel/SongEmotionDataset/100.mp3'), PosixPat

[tensor([8., 0., 2., 3., 9., 2., 5., 2.], device='cuda:0'), tensor([1., 0., 0., 2., 6., 6., 6., 1.], device='cuda:0'), tensor([1., 8., 5., 8., 0., 0., 0., 4.], device='cuda:0'), tensor([8., 1., 6., 4., 1., 2., 6., 5.], device='cuda:0'), tensor([1., 7., 4., 5., 0., 2., 1., 1.], device='cuda:0'), tensor([ 1.,  0.,  1.,  0., 10.,  6.,  1.,  0.], device='cuda:0'), tensor([ 8.,  1.,  6., 11.,  6., 19., 33.,  3.], device='cuda:0'), tensor([ 1.,  4.,  3., 12.,  0.,  1.,  2.,  3.], device='cuda:0'), tensor([ 5., 21., 21., 21.,  0.,  1.,  3., 17.], device='cuda:0'), tensor([ 4.,  7.,  4.,  5., 10., 38.,  7.,  1.], device='cuda:0'), tensor([10., 10., 14., 26.,  3., 11.,  6.,  6.], device='cuda:0'), tensor([4., 3., 4., 1., 3., 5., 4., 5.], device='cuda:0'), tensor([10.,  2.,  2.,  2.,  5.,  1.,  8., 11.], device='cuda:0'), tensor([2., 1., 5., 5., 1., 5., 2., 4.], device='cuda:0'), tensor([ 5.,  3.,  7.,  4., 21., 21., 16.,  3.], device='cuda:0'), tensor([10., 13., 21., 11.,  2.,  6.,  7.,  9.], d

In [8]:
train_emotion

[tensor([8., 0., 2., 3., 9., 2., 5., 2.], device='cuda:0'),
 tensor([1., 0., 0., 2., 6., 6., 6., 1.], device='cuda:0'),
 tensor([1., 8., 5., 8., 0., 0., 0., 4.], device='cuda:0'),
 tensor([8., 1., 6., 4., 1., 2., 6., 5.], device='cuda:0'),
 tensor([1., 7., 4., 5., 0., 2., 1., 1.], device='cuda:0'),
 tensor([ 1.,  0.,  1.,  0., 10.,  6.,  1.,  0.], device='cuda:0'),
 tensor([ 8.,  1.,  6., 11.,  6., 19., 33.,  3.], device='cuda:0'),
 tensor([ 1.,  4.,  3., 12.,  0.,  1.,  2.,  3.], device='cuda:0'),
 tensor([ 5., 21., 21., 21.,  0.,  1.,  3., 17.], device='cuda:0'),
 tensor([ 4.,  7.,  4.,  5., 10., 38.,  7.,  1.], device='cuda:0'),
 tensor([10., 10., 14., 26.,  3., 11.,  6.,  6.], device='cuda:0'),
 tensor([4., 3., 4., 1., 3., 5., 4., 5.], device='cuda:0'),
 tensor([10.,  2.,  2.,  2.,  5.,  1.,  8., 11.], device='cuda:0'),
 tensor([2., 1., 5., 5., 1., 5., 2., 4.], device='cuda:0'),
 tensor([ 5.,  3.,  7.,  4., 21., 21., 16.,  3.], device='cuda:0'),
 tensor([10., 13., 21., 11.,  2.,  6

In [9]:
class SongEmotionDataset(Dataset):
    """
    Song Emotion Dataset. Uses librosa to process mp3 files.
    Takes first 20 seconds, and samples every 10 to get processed audio tensor.
    """

    def __init__(self, mp3, labels, transform=None):
        """
        Args:
            mp3: list of paths to mp3 files
            labels: list of labels
        """
        self.labels = labels
        self.mp3 = mp3
        self.cache = {}
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        if index not in self.cache.keys():
#             print("index of " + str(index) + " was cached!")
            data, rate = librosa.load(self.mp3[index], sr=16000, duration=10)
            assert rate == 16000
            sample_tensor = torch.tensor(data, device=device).float()
            downsampled_tensor = sample_tensor[::10]

            self.cache[index] = (downsampled_tensor, F.softmax(self.labels[index]))
#         else:
#             print("index was cached! index of " + str(index))
#         print(self.cache[index])
        return self.cache[index]

        
#         data, rate = librosa.load(self.mp3[index], sr=16000, duration=20)
#         assert rate == 16000
#         sample_tensor = torch.tensor(data, device=device).float()
#         downsampled_tensor = sample_tensor[::10]
        
#         print(downsampled_tensor, F.softmax(self.labels[index]))
        
#         return downsampled_tensor, F.softmax(self.labels[index])

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 128, 80, 4)
        self.bn1 = nn.BatchNorm1d(128)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(128, 128, 3)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(128, 256, 3)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(256, 512, 3)
        self.bn4 = nn.BatchNorm1d(512)
        self.pool4 = nn.MaxPool1d(4)
        self.avgPool = nn.AvgPool1d(14) #input should be 512x30 so this outputs a 512x1
        self.fc1 = nn.Linear(512, 8)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = self.avgPool(x)
        x = x.permute(0, 2, 1) #change the 512x1 to 1x512
        x = self.fc1(x)
        return F.log_softmax(x, dim = 2)

model = Net()
model.to(device)
print(model)

Net(
  (conv1): Conv1d(1, 128, kernel_size=(80,), stride=(4,))
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(256, 512, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (avgPool): AvgPool1d(kernel_size=(14,), stride=(14,), 

In [11]:
train_set = SongEmotionDataset(train_song, train_emotion)
test_set = SongEmotionDataset(test_song, test_emotion)
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 8, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 8, shuffle = True, **kwargs)

optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)

Train set size: 180
Test set size: 45


In [12]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data.unsqueeze_(1)
        data = data.requires_grad_() #set requires_grad to True for training
        output = model(data)
        output = output.view(-1, len(emotions))
#         print(output.shape, target.shape)
#         print(output, target)
        loss = F.kl_div(output, target)
        loss.backward()
        optimizer.step()
#         scheduler.step()
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))

In [13]:
def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:
        data.unsqueeze_(1)
        output = model(data)
        output = output.permute(1, 0, 2)
        print(output)
        pred = output.max(2)[1] # get the index of the max log-probability
        correct += pred.eq(target.max(1)[1]).cpu().sum().item()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [14]:
#w caching & data processing
import warnings

log_interval = 5
warnings.filterwarnings("ignore")
for epoch in range(1, 41):
    print("training epoch " + str(epoch))
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
#     scheduler.step()
    train(model, epoch)
    scheduler.step()
    test(model, epoch)

training epoch 1
Train Epoch: 1 [0/180 (0%)]	Loss: 0.243077
Train Epoch: 1 [40/180 (22%)]	Loss: 0.536850
Train Epoch: 1 [80/180 (43%)]	Loss: 0.404568
Train Epoch: 1 [120/180 (65%)]	Loss: 0.358676
Train Epoch: 1 [160/180 (87%)]	Loss: 0.314944
tensor([[[-2.5798, -2.8025, -2.1743, -1.4982, -2.7472, -1.9977, -1.3769,
          -2.6005],
         [-2.8653, -3.2110, -2.2288, -1.2844, -3.0877, -1.7916, -1.3926,
          -2.8556],
         [-2.8545, -3.2006, -2.2241, -1.2912, -3.0833, -1.7981, -1.3909,
          -2.8420],
         [-2.7149, -3.0014, -2.1979, -1.3885, -2.9238, -1.9029, -1.3661,
          -2.7236],
         [-3.2036, -3.7208, -2.2943, -1.1254, -3.4915, -1.4945, -1.5594,
          -3.1065],
         [-2.9359, -3.3287, -2.2332, -1.2492, -3.1860, -1.7092, -1.4285,
          -2.8978],
         [-2.8061, -3.1311, -2.2139, -1.3233, -3.0214, -1.8347, -1.3815,
          -2.8010],
         [-2.7486, -3.0524, -2.2037, -1.3631, -2.9700, -1.8774, -1.3686,
          -2.7531]]], device='cuda

Train Epoch: 3 [40/180 (22%)]	Loss: 0.242236
Train Epoch: 3 [80/180 (43%)]	Loss: 0.174477
Train Epoch: 3 [120/180 (65%)]	Loss: 0.175387
Train Epoch: 3 [160/180 (87%)]	Loss: 0.178412
tensor([[[-2.0222, -1.8210, -1.5980, -3.4422, -3.2038, -1.7935, -1.9810,
          -2.0672],
         [-1.8124, -2.0373, -2.0804, -2.7005, -3.0670, -1.6911, -1.7250,
          -2.2514],
         [-2.3060, -2.4005, -1.8262, -2.7777, -2.3577, -1.9404, -1.4775,
          -2.1206],
         [-1.9872, -2.3729, -2.1187, -2.2869, -2.5949, -1.7945, -1.5640,
          -2.3251],
         [-1.8627, -2.6691, -2.4528, -2.1077, -2.3437, -2.0677, -1.3153,
          -2.5638],
         [-1.8946, -2.9210, -2.6226, -2.1725, -2.1327, -2.3419, -1.1149,
          -2.7089],
         [-1.9017, -2.7982, -2.5118, -2.3719, -2.3623, -2.1969, -1.1035,
          -2.5476],
         [-1.7755, -2.3536, -2.1083, -2.7775, -2.8587, -1.9407, -1.3543,
          -2.3765]]], device='cuda:0', grad_fn=<PermuteBackward>)
tensor([[[-2.2585, -2.6214, 

tensor([[[-2.1101, -2.2351, -1.9209, -2.3372, -1.6428, -2.0057, -2.1003,
          -2.5474],
         [-2.2771, -2.0484, -2.4148, -2.0999, -2.3233, -1.8745, -1.6473,
          -2.1832],
         [-2.1357, -2.1965, -1.9223, -2.2764, -1.7004, -2.0677, -1.9973,
          -2.5648],
         [-2.1624, -2.4311, -2.6283, -1.7910, -2.2708, -1.6756, -1.7481,
          -2.3701],
         [-2.1201, -2.5395, -2.6692, -1.8086, -2.1239, -1.7395, -1.7108,
          -2.3861],
         [-2.2834, -2.2403, -2.6194, -1.7125, -2.5900, -1.5252, -1.8666,
          -2.3954],
         [-2.2935, -2.0996, -2.5356, -1.7737, -2.6331, -1.5533, -1.9123,
          -2.3381],
         [-2.1665, -2.3993, -2.6488, -1.7957, -2.3145, -1.6613, -1.7417,
          -2.3646]]], device='cuda:0', grad_fn=<PermuteBackward>)
tensor([[[-2.0730, -2.5135, -1.8075, -1.7979, -1.9601, -1.9946, -2.5667,
          -2.2091],
         [-2.1866, -2.2712, -2.5335, -1.7873, -2.4159, -1.6023, -1.8927,
          -2.3399],
         [-2.1002, -2.57

Train Epoch: 7 [40/180 (22%)]	Loss: 0.154355
Train Epoch: 7 [80/180 (43%)]	Loss: 0.155033
Train Epoch: 7 [120/180 (65%)]	Loss: 0.161189
Train Epoch: 7 [160/180 (87%)]	Loss: 0.235141
tensor([[[-2.1575, -1.9978, -2.4490, -2.6011, -2.1035, -2.3474, -1.4928,
          -1.9255],
         [-2.0803, -2.3351, -2.4334, -1.8053, -2.3102, -2.0261, -1.7526,
          -2.1058],
         [-1.9447, -2.5013, -2.6650, -2.0328, -1.9325, -2.3703, -1.5430,
          -2.1001],
         [-1.9804, -2.2035, -2.3846, -2.1950, -2.2446, -2.2580, -1.7386,
          -1.8214],
         [-1.9355, -2.3816, -2.5404, -2.1768, -2.0286, -2.3731, -1.6087,
          -1.9232],
         [-2.0302, -2.4437, -2.5613, -1.8172, -2.1578, -2.1260, -1.6520,
          -2.1575],
         [-2.1276, -1.9153, -2.5095, -2.2927, -2.0802, -2.2454, -1.6014,
          -2.1298],
         [-2.0378, -2.2848, -2.4243, -1.9266, -2.3096, -2.0863, -1.7409,
          -2.0032]]], device='cuda:0', grad_fn=<PermuteBackward>)
tensor([[[-2.1701, -1.9797, 

tensor([[[-1.8870, -2.4249, -2.6488, -2.1213, -2.2183, -2.5797, -1.4374,
          -1.9158],
         [-2.4313, -2.7113, -2.5695, -1.3140, -2.2946, -1.9458, -1.7641,
          -2.4614],
         [-2.0248, -2.5480, -2.6323, -1.7153, -2.3251, -2.2223, -1.5399,
          -2.1422],
         [-2.5311, -2.7587, -2.4335, -1.2046, -2.2063, -2.0977, -1.8756,
          -2.4839],
         [-2.4176, -2.7215, -2.6327, -1.3280, -2.5305, -1.8611, -1.6612,
          -2.4869],
         [-2.5742, -2.5680, -2.1877, -1.2683, -2.0043, -2.1243, -2.2405,
          -2.3758],
         [-2.3117, -2.7018, -2.6509, -1.4286, -2.3669, -1.9681, -1.5914,
          -2.4479],
         [-5.9473, -4.8937, -3.2515, -0.1307, -3.4253, -3.4460, -6.3880,
          -4.8795]]], device='cuda:0', grad_fn=<PermuteBackward>)
tensor([[[-2.6526, -2.5224, -2.1903, -1.2659, -2.0379, -1.9995, -2.2798,
          -2.4374],
         [-2.3776, -2.7395, -2.6379, -1.3232, -2.4920, -1.8992, -1.6753,
          -2.4635],
         [-2.5050, -2.54

...

Train Epoch: 31 [40/180 (22%)]	Loss: 0.095863
Train Epoch: 31 [80/180 (43%)]	Loss: 0.123416
Train Epoch: 31 [120/180 (65%)]	Loss: 0.101685
Train Epoch: 31 [160/180 (87%)]	Loss: 0.124940
tensor([[[-6.5784, -2.5558, -1.5856, -3.2491, -4.9620, -0.4105, -7.4258,
          -5.0500],
         [-4.3592, -1.6408, -0.9469, -2.8730, -3.5746, -1.4965, -4.3469,
          -2.4766],
         [-3.4196, -2.3816, -1.7121, -1.5858, -3.5910, -1.0603, -3.0643,
          -2.6740],
         [-2.0166, -2.4526, -2.3540, -1.6657, -1.9267, -2.3661, -1.6210,
          -2.8201],
         [-2.7696, -1.6920, -1.2922, -2.6569, -1.9038, -2.6365, -2.2691,
          -2.4732],
         [-5.4736, -2.7892, -0.5571, -6.1391, -1.2541, -4.7871, -2.8472,
          -4.8759],
         [-4.1826, -1.4241, -0.8528, -3.5129, -1.8554, -2.4150, -4.5255,
          -3.4609],
         [-2.5032, -2.3438, -2.0389, -1.3595, -2.8318, -1.8244, -2.1862,
          -2.2762]]], device='cuda:0', grad_fn=<PermuteBackward>)
tensor([[[-3.3424, -2.00

tensor([[[-3.0936, -2.2802, -1.8621, -1.6046, -3.0784, -1.1887, -2.6767,
          -2.5687],
         [-6.9225, -7.2049, -5.3010, -0.6919, -8.5803, -0.7093, -7.7404,
          -9.7172],
         [-2.7691, -3.1042, -2.6187, -0.9576, -3.0951, -1.3789, -2.2785,
          -3.3203],
         [-2.6579, -2.6145, -1.9382, -3.1538, -1.1657, -3.0625, -1.2915,
          -3.3044],
         [-2.0246, -2.8675, -2.5804, -3.3558, -3.0432, -5.5529, -1.0741,
          -1.1799],
         [-2.3796, -2.1690, -1.8471, -1.3806, -2.7554, -1.9871, -2.3820,
          -2.3975],
         [-2.4510, -2.0042, -1.9582, -1.8364, -1.7682, -1.7714, -2.4232,
          -3.0130],
         [-1.9625, -3.0879, -3.0542, -1.6878, -1.6281, -2.7722, -1.2436,
          -3.3629]]], device='cuda:0', grad_fn=<PermuteBackward>)
tensor([[[ -1.7852,  -2.5302,  -2.5223,  -1.7135,  -2.0443,  -2.9785,  -1.5436,
           -2.3214],
         [ -6.4300,  -8.1250,  -6.3052,  -0.3379,  -8.4139,  -1.2680,  -6.6176,
          -10.4765],
        

Train Epoch: 35 [40/180 (22%)]	Loss: 0.126010
Train Epoch: 35 [80/180 (43%)]	Loss: 0.072106
Train Epoch: 35 [120/180 (65%)]	Loss: 0.110038
Train Epoch: 35 [160/180 (87%)]	Loss: 0.130730
tensor([[[ -6.2608,  -4.5802,  -3.3700,  -2.3190,  -7.0368,  -0.1651,  -5.8142,
           -5.6802],
         [ -1.6354,  -3.2770,  -3.1207,  -1.6732,  -1.8423,  -3.1057,  -1.1868,
           -3.6059],
         [ -4.7463,  -2.9681,  -1.5124,  -2.7595,  -4.0362,  -0.4871,  -4.5406,
           -4.3082],
         [ -2.8772,  -3.9309,  -3.2625,  -0.3819,  -3.9566,  -1.9901,  -3.2133,
           -4.9331],
         [ -7.2862,  -8.2454,  -6.0322,  -0.5685,  -9.5482,  -0.8441,  -8.2397,
          -11.2538],
         [ -2.0543,  -3.1087,  -2.4543,  -3.1861,  -2.0188,  -4.2663,  -0.7119,
           -2.7754],
         [ -1.9396,  -2.5190,  -2.4066,  -1.4274,  -2.5633,  -2.5288,  -1.7269,
           -2.1984],
         [ -2.6919,  -1.9038,  -1.3354,  -3.3864,  -3.0624,  -4.8980,  -3.2310,
           -0.9349]]], devi

tensor([[[-1.3937, -3.2816, -3.0903, -0.9395, -2.9166, -3.7009, -1.9165,
          -2.9559],
         [-2.2255, -2.4510, -2.1395, -1.6042, -2.0067, -2.2768, -1.7210,
          -2.6439],
         [-1.8041, -3.0492, -3.0517, -1.8141, -1.7095, -3.1510, -1.1828,
          -3.0462],
         [-5.1588, -2.0895, -1.0713, -3.1895, -4.2853, -0.8816, -4.6692,
          -3.0061],
         [-2.2070, -2.3301, -2.0758, -1.5714, -2.2970, -2.2087, -1.8614,
          -2.3686],
         [-1.9301, -3.1150, -3.0320, -1.9575, -1.5673, -3.2959, -1.0969,
          -3.1815],
         [-2.5153, -2.4097, -1.7322, -1.2224, -2.1222, -1.8220, -3.0182,
          -3.5920],
         [-2.3578, -1.9087, -1.5186, -2.3700, -1.9369, -3.1707, -2.1662,
          -1.9389]]], device='cuda:0', grad_fn=<PermuteBackward>)
tensor([[[-2.4411, -2.9031, -2.4912, -1.0960, -2.7545, -1.7991, -1.8546,
          -2.8917],
         [-1.9085, -2.9831, -2.9216, -1.8427, -1.6705, -2.9716, -1.1972,
          -3.0484],
         [-2.9975, -2.44

Train Epoch: 39 [40/180 (22%)]	Loss: 0.156596
Train Epoch: 39 [80/180 (43%)]	Loss: 0.117393
Train Epoch: 39 [120/180 (65%)]	Loss: 0.106539
Train Epoch: 39 [160/180 (87%)]	Loss: 0.123461
tensor([[[-1.8509, -2.4580, -2.2833, -1.6812, -2.0409, -2.9633, -1.6008,
          -2.4546],
         [-1.7690, -4.2867, -4.0532, -0.9749, -1.9935, -3.3895, -1.4037,
          -5.1949],
         [-1.6864, -4.0310, -3.2495, -0.8932, -1.5166, -3.3941, -2.3802,
          -5.6962],
         [-3.6865, -1.7308, -0.7680, -4.2123, -1.7665, -5.0276, -4.5941,
          -2.0293],
         [-3.4965, -3.3107, -1.7975, -5.3405, -2.8725, -5.6266, -0.6092,
          -1.8405],
         [-2.9864, -1.8328, -1.6616, -3.2429, -0.8633, -2.8576, -3.1558,
          -3.2468],
         [-1.8576, -3.0021, -2.9863, -1.7945, -1.6651, -2.8250, -1.2425,
          -3.2083],
         [-6.5369, -1.8265, -0.9400, -6.2393, -0.8512, -4.2479, -8.8633,
          -5.6112]]], device='cuda:0', grad_fn=<PermuteBackward>)
tensor([[[-2.5321, -2.31

In [67]:
#W/ CACHING
import warnings

log_interval = 5
warnings.filterwarnings("ignore")
for epoch in range(1, 41):
    print("training epoch " + str(epoch))
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
#     scheduler.step()
    train(model, epoch)
    scheduler.step()
    test(model, epoch)

training epoch 1
Train Epoch: 1 [0/320 (0%)]	Loss: 0.191720
Train Epoch: 1 [40/320 (12%)]	Loss: 0.329261
Train Epoch: 1 [80/320 (25%)]	Loss: 0.284903
Train Epoch: 1 [120/320 (38%)]	Loss: 0.338861
Train Epoch: 1 [160/320 (50%)]	Loss: 0.210636
Train Epoch: 1 [200/320 (62%)]	Loss: 0.167495
Train Epoch: 1 [240/320 (75%)]	Loss: 0.162565
Train Epoch: 1 [280/320 (88%)]	Loss: 0.321939

Test set: Accuracy: 19/80 (24%)

training epoch 2
Train Epoch: 2 [0/320 (0%)]	Loss: 0.210194
Train Epoch: 2 [40/320 (12%)]	Loss: 0.201684
Train Epoch: 2 [80/320 (25%)]	Loss: 0.117230
Train Epoch: 2 [120/320 (38%)]	Loss: 0.207198
Train Epoch: 2 [160/320 (50%)]	Loss: 0.188115
Train Epoch: 2 [200/320 (62%)]	Loss: 0.190015
Train Epoch: 2 [240/320 (75%)]	Loss: 0.140593
Train Epoch: 2 [280/320 (88%)]	Loss: 0.213586

Test set: Accuracy: 18/80 (22%)

training epoch 3
Train Epoch: 3 [0/320 (0%)]	Loss: 0.127676
Train Epoch: 3 [40/320 (12%)]	Loss: 0.175529
Train Epoch: 3 [80/320 (25%)]	Loss: 0.160134
Train Epoch: 3 [120/32

Train Epoch: 20 [280/320 (88%)]	Loss: 0.164417

Test set: Accuracy: 30/80 (38%)

training epoch 21
Train Epoch: 21 [0/320 (0%)]	Loss: 0.088417
Train Epoch: 21 [40/320 (12%)]	Loss: 0.127397
Train Epoch: 21 [80/320 (25%)]	Loss: 0.162288
Train Epoch: 21 [120/320 (38%)]	Loss: 0.132321
Train Epoch: 21 [160/320 (50%)]	Loss: 0.123884
Train Epoch: 21 [200/320 (62%)]	Loss: 0.132901
Train Epoch: 21 [240/320 (75%)]	Loss: 0.092551
Train Epoch: 21 [280/320 (88%)]	Loss: 0.152662

Test set: Accuracy: 29/80 (36%)

training epoch 22
Train Epoch: 22 [0/320 (0%)]	Loss: 0.099070
Train Epoch: 22 [40/320 (12%)]	Loss: 0.107604
Train Epoch: 22 [80/320 (25%)]	Loss: 0.164107
Train Epoch: 22 [120/320 (38%)]	Loss: 0.155712
Train Epoch: 22 [160/320 (50%)]	Loss: 0.088193
Train Epoch: 22 [200/320 (62%)]	Loss: 0.099478
Train Epoch: 22 [240/320 (75%)]	Loss: 0.130642
Train Epoch: 22 [280/320 (88%)]	Loss: 0.101703

Test set: Accuracy: 29/80 (36%)

training epoch 23
Train Epoch: 23 [0/320 (0%)]	Loss: 0.141142
Train Epoch

Train Epoch: 40 [120/320 (38%)]	Loss: 0.133885
Train Epoch: 40 [160/320 (50%)]	Loss: 0.145604
Train Epoch: 40 [200/320 (62%)]	Loss: 0.060129
Train Epoch: 40 [240/320 (75%)]	Loss: 0.087228
Train Epoch: 40 [280/320 (88%)]	Loss: 0.111683

Test set: Accuracy: 30/80 (38%)



In [ ]:
#ORIGINAL
import warnings

log_interval = 5
warnings.filterwarnings("ignore")
for epoch in range(1, 41):
    print("training epoch " + str(epoch))
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
#     scheduler.step()
    train(model, epoch)
    scheduler.step()
    test(model, epoch)

training epoch 1
Train Epoch: 1 [0/320 (0%)]	Loss: 0.238086
Train Epoch: 1 [40/320 (12%)]	Loss: 0.543195
Train Epoch: 1 [80/320 (25%)]	Loss: 0.277862
Train Epoch: 1 [120/320 (38%)]	Loss: 0.275000
Train Epoch: 1 [160/320 (50%)]	Loss: 0.128894
Train Epoch: 1 [200/320 (62%)]	Loss: 0.191396
Train Epoch: 1 [240/320 (75%)]	Loss: 0.220283
Train Epoch: 1 [280/320 (88%)]	Loss: 0.195243

Test set: Accuracy: 21/80 (26%)

training epoch 2
Train Epoch: 2 [0/320 (0%)]	Loss: 0.170415
Train Epoch: 2 [40/320 (12%)]	Loss: 0.197089
Train Epoch: 2 [80/320 (25%)]	Loss: 0.203403
Train Epoch: 2 [120/320 (38%)]	Loss: 0.202500
Train Epoch: 2 [160/320 (50%)]	Loss: 0.189248
Train Epoch: 2 [200/320 (62%)]	Loss: 0.190571
Train Epoch: 2 [240/320 (75%)]	Loss: 0.205890
Train Epoch: 2 [280/320 (88%)]	Loss: 0.166318

Test set: Accuracy: 27/80 (34%)

training epoch 3
Train Epoch: 3 [0/320 (0%)]	Loss: 0.150036
Train Epoch: 3 [40/320 (12%)]	Loss: 0.133074
Train Epoch: 3 [80/320 (25%)]	Loss: 0.141485
Train Epoch: 3 [120/32

Train Epoch: 20 [200/320 (62%)]	Loss: 0.127817
Train Epoch: 20 [240/320 (75%)]	Loss: 0.105206
Train Epoch: 20 [280/320 (88%)]	Loss: 0.118119

Test set: Accuracy: 24/80 (30%)

training epoch 21
Train Epoch: 21 [0/320 (0%)]	Loss: 0.120037
Train Epoch: 21 [40/320 (12%)]	Loss: 0.187839
Train Epoch: 21 [80/320 (25%)]	Loss: 0.096697
Train Epoch: 21 [120/320 (38%)]	Loss: 0.168122
Train Epoch: 21 [160/320 (50%)]	Loss: 0.106986
Train Epoch: 21 [200/320 (62%)]	Loss: 0.132516
Train Epoch: 21 [240/320 (75%)]	Loss: 0.094457
Train Epoch: 21 [280/320 (88%)]	Loss: 0.115992

Test set: Accuracy: 19/80 (24%)

training epoch 22
Train Epoch: 22 [0/320 (0%)]	Loss: 0.114777
Train Epoch: 22 [40/320 (12%)]	Loss: 0.126823
Train Epoch: 22 [80/320 (25%)]	Loss: 0.130811
Train Epoch: 22 [120/320 (38%)]	Loss: 0.103881
Train Epoch: 22 [160/320 (50%)]	Loss: 0.136740
Train Epoch: 22 [200/320 (62%)]	Loss: 0.145961
Train Epoch: 22 [240/320 (75%)]	Loss: 0.172568
Train Epoch: 22 [280/320 (88%)]	Loss: 0.130506

Test set: Ac

## NOTES

below is the mfccs notes / random code

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])
torch.save(model.state_dict(), 'dataset_model_soundemotion.pt')

In [8]:
# audio, sample_rate = librosa.load("SongEmotionDataset/1.mp3", res_type='kaiser_fast')
# # [print(x) for x in audio]

# #convert audio into 2d array
# mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
# # mfccsscaled = np.mean(mfccs.T,axis=0)
# print(mfccs.shape, audio.shape)
# mfccs

In [10]:
# audio_tensor = torch.tensor(audio)
# audio_tensor
# audio_tensor.shape

In [16]:
# for sound_file in data_path.iterdir():
#     if ".mp3" in str(sound_file):
#         print(sound_file)
#         audio, sample_rate = librosa.load(str(sound_file), res_type='kaiser_fast')
        
    

In [15]:
# train_percentage = 0.8
# allowed_exceedance = 0

# train_song = []
# test_song = []
# train_emotion = []
# test_emotion = []

# row_indexes = np.arange(1,401)
# np.random.shuffle(row_indexes)

# # train_indexes = [row_indexes[i] for i in range(len(row_indexes)) if i < len(row_indexes)*train_percentage]
# # test_indexes = [row_indexes[i] for i in range(len(row_indexes)) if i >= len(row_indexes)*train_percentage]

# def get_data(indexes):
#     song = []
#     emotion = []
    
#     totals = torch.zeros(len(emotions), device=device).float()
#     for x in indexes:    
#         row = torch.tensor([sheet.cell_value(x, 2 + j) for j in range(8)], device=device).float()
#         totals += F.softmax(row)

#     min_total = torch.min(totals)
#     print(totals)
    
#     totals = torch.zeros(len(emotions), device=device).float()
#     for x in indexes:
#         row = torch.tensor([sheet.cell_value(x, 2 + j) for j in range(8)], device=device).float()
        
#         if torch.max(totals + row) < min_total*(1 + allowed_exceedance):
#             song.append(dataset.joinpath("{}.mp3".format(x)))
#             emotion.append(row)
#             totals += F.softmax(row)
            
#     print(totals)
#     return song, emotion
    
# song, emotion = get_data(row_indexes)
# # test_song, test_emotion = get_data(test_indexes)

# train_song = [song[i] for i in range(len(song)) if i < len(song)*train_percentage]
# test_song = [song[i] for i in range(len(song)) if i >= len(song)*train_percentage]

# train_emotion = [emotion[i] for i in range(len(emotion)) if i < len(emotion)*train_percentage]
# test_emotion = [emotion[i] for i in range(len(emotion)) if i >= len(emotion)*train_percentage]

# num_maxes = [0 for _ in emotions]

# for row in emotion:
#     i = torch.argmax(row)
#     num_maxes[i] += 1
    
# print(num_maxes)

tensor([ 11.4343, 157.9054,  55.5535, 120.1633,  54.9434], device='cuda:0')
tensor([3.1112, 9.2133, 7.9749, 8.9079, 7.7927], device='cuda:0')
[1, 9, 9, 11, 7]


In [16]:
# train_song = []
# test_song = []
# train_emotion = []
# test_emotion = []

# for i in range(1, 401):
#     count_total = sheet.cell_value(i, 11)
#     if i % 5 == 0:
#         test_song.append(dataset.joinpath("{}.mp3".format(i)))
#         emotion_arr = []
#         for j in range(9):
#             emotion_arr.append(sheet.cell_value(i, 2 + j))
#         test_emotion.append(torch.tensor(emotion_arr, device=device).float())
#     else:
#         train_song.append(dataset.joinpath("{}.mp3".format(i)))
#         emotion_arr = []
#         for j in range(9):
#             emotion_arr.append(sheet.cell_value(i, 2 + j))
#         train_emotion.append(torch.tensor(emotion_arr, device=device))

# print(len(train_song), len(test_song))
# print(len(train_emotion), len(test_emotion))